In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/My Drive/DataSet/DatasetTiles

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/DataSet/DatasetTiles


In [6]:
import os
import time
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
from torch import autograd, optim
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import cv2